In [1]:
import scipy
import numpy as np
import pandas as pd
import plotly.plotly as py
# import visplots
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from sklearn import preprocessing, metrics
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from scipy.stats.distributions import randint
import csv
import numpy as np
from pandas import Series, DataFrame
import plotly.graph_objs as go
from plotly import tools

from nltk import word_tokenize, wordpunct_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs
from nltk.classify.api import ClassifierI

# getting around the ascii characters
from django.utils.encoding import smart_str, smart_unicode

In [2]:
# Import the clean tweets (text and date)
twitter_raw = pd.read_csv("C:\\Data\\Twitter\\Tweets\\Clean_Tweets1.csv", sep=',', delimiter=None)

# Delete tweets containing key words

In [3]:
twitter_cleaned=twitter_raw[twitter_raw['text'].str.lower().str.contains \
                   ("great smog of london|spanish|sex|porn|anal|pov|bbw|milf|sexy|shemale|sexyfishrestaurant|\
                   nude|sluts|super hot blonde|adult video|erotic|18+|dirty fun|killer fog|rihanna")==False]

twitter_cleaned = twitter_cleaned.reset_index(drop=True)

twitter_cleaned['text'].count()

9419

#  Machine Learning to clean tweets

# 1. Create the Maching learning

In [4]:
twlist = []

with open(r"C:\\Git\\Weather\\trained_weather_NoRT.csv", "r") as t:
    tweets_raw = pd.read_csv(t)

tweets = tweets_raw[['text', 'weather']].values.tolist()

twlist = [tuple(l) for l in tweets] # turn nested list of lists into list of tuples
twtokens = []

In [5]:
for (words, weather) in twlist:
#    words_filtered = [e.lower().decode('utf8') for e in words.split() if len(e) >= 3 and len(e) <= 10] # and <= 10
    words_filtered = [unicode(e.lower(), errors = 'replace') for e in words.split() if len(e) >= 3 and len(e) <= 10] # and <= 10
    twtokens.append((words_filtered, weather))

In [6]:
import nltk
def get_words_in_tweets(tweets):
    all_words = []
    for (words, weather) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

In [7]:
word_features = get_word_features(get_words_in_tweets(twtokens))

In [8]:
# The Classifier
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [9]:
training_set = nltk.classify.apply_features(extract_features, tweets)

# train the classifier
classifier = nltk.NaiveBayesClassifier.train(training_set)

print classifier.show_most_informative_features(32)

Most Informative Features
    contains(summer���s) = False              no : yes    =      1.0 : 1.0
           contains(66%) = False              no : yes    =      1.0 : 1.0
      contains(forecast) = False              no : yes    =      1.0 : 1.0
     contains(@thedoors) = False              no : yes    =      1.0 : 1.0
      contains(@pc24fym) = False              no : yes    =      1.0 : 1.0
    contains(infinit...) = False              no : yes    =      1.0 : 1.0
       contains(finally) = False              no : yes    =      1.0 : 1.0
        contains(1012mb) = False              no : yes    =      1.0 : 1.0
       contains(provide) = False              no : yes    =      1.0 : 1.0
    contains(@moosawi17) = False              no : yes    =      1.0 : 1.0
      contains(mentally) = False              no : yes    =      1.0 : 1.0
          contains(#now) = False              no : yes    =      1.0 : 1.0
         contains(rainy) = False              no : yes    =      1.0 : 1.0

# 2. Classify London Tweets

In [10]:
df = pd.DataFrame(twitter_cleaned)

tx = df['text']
df['text'] = tx

In [11]:
dffinal = df[['text']]
dffinal['date'] = df['Date']

In [12]:
for i in range(len(df.index)):
    if classifier.classify(extract_features((smart_str(df['text'][i])).split())) == 'yes':
        dffinal['text'][i] = smart_str(df['text'][i])
    else:
        dffinal['text'][i] = None

count = 0
for i in range(len(df.index)):
    if dffinal['text'][i] != None:
        count += 1

count2 = 0
dffinaltrained = pd.DataFrame({'date' : pd.Series(range(count), index=range(count)), 'text' : pd.Series(range(count), index=range(count))})
for i in range(len(df.index)):
    if dffinal['text'][i] != None:
        dffinaltrained['text'][count2] = dffinal['text'][i]
        dffinaltrained['date'][count2] = dffinal['date'][i]
        count2 += 1

C:\Users\Student29\Anaconda2\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Student29\Anaconda2\lib\site-packages\pandas\core\indexing.py:140: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [13]:
dffinaltrained['text'].count()

8528

# Define function which will count the weather words

In [14]:
def wordCount(tweet):
    # List of words we are looking at
    weather_words = ['breeze', 'breezy', 'cloudy', 'cold', 'ice', 'icy', 'icey', 'drizzle', 'frost', 'wind', 'mild', 'dew', 
                     'freezing', 'downpour', 'shower', 'rain', 'frost', 'nippy', 'hail', 'temperature', 'gail', 'gust',
                     'sleet', 'heat', 'storm', 'slush', 'fog', 'foggy', 'flood', 'visibility', 'warm', 'warmer',
                     'mist', 'frosty', 'misty', 'chilly', 'thunder', 'lightning', 'snow', 'snowing', 'hot', 'sun', 'sunny',
                     'boiling', 'baltic', 'burn']
    # Create a new dictionnary
    counts = dict()
    for t in tweet:
        if t in weather_words:
            counts[t] = counts.get(t,0) + 1

    return counts

# Get the number of final clean tweets per day

In [15]:
# Check the days and number of tweets per day
dffinaltrained.groupby(['date']).count()

,text
date,
2016-12-14,300
2016-12-15,255
2016-12-16,795
2016-12-19,791
2016-12-20,804
2016-12-21,421
2016-12-22,463
2016-12-23,438
2016-12-24,447


In [16]:
# Create a new dataframe with the date
tw_date = dffinaltrained.groupby(['date']).count()
tw_date.index

Index([u'2016-12-14', u'2016-12-15', u'2016-12-16', u'2016-12-19',
       u'2016-12-20', u'2016-12-21', u'2016-12-22', u'2016-12-23',
       u'2016-12-24', u'2016-12-25', u'2016-12-26', u'2016-12-27',
       u'2016-12-28', u'2016-12-30', u'2016-12-31', u'2017-01-01',
       u'2017-01-02', u'2017-01-03'],
      dtype='object', name=u'date')

# Going through all tweets then count words and put them in final DataFrame (per day)

In [18]:
# Initialise the final dataframe
tw_final = DataFrame()
tw_final['date'] = tw_date.index

tw_final['Keyword_1'] = None
tw_final['Keyword_2'] = None
tw_final['Keyword_3'] = None
tw_final['Keyword_4'] = None
tw_final['Keyword_5'] = None

In [19]:
for i in range(len(tw_date.index)):
    df = dffinaltrained[(dffinaltrained['date'] == tw_final['date'][i])]
    twitterlist = []
    for t in range(len(df.index)):
        twitterlist.append(dffinaltrained['text'][t].lower().split())

    # Flatten the list so all values in the same list
    tweets_flatten = [j for sublist in twitterlist for j in sublist]

    # Counting the number of key words in the tweets
    d = wordCount(tweets_flatten)
    
    # give a list of the sorted tweets, with most popular coming first
    d2 = sorted(d.items(), key = lambda x: x[1], reverse = True)

    
    tw_final['Keyword_1'][i] = d2[0]#[0]  #keyword 1 is the most popular
    tw_final['Keyword_2'][i] = d2[1]#[0]  #keyword 2 is the second most popular
    tw_final['Keyword_3'][i] = d2[2]#[0]
    tw_final['Keyword_4'][i] = d2[3]#[0]
    tw_final['Keyword_5'][i] = d2[4]#[0]

In [71]:
# Making things simpler
tw_final = tw_final.drop(['Keyword_4','Keyword_5'], axis=1)
tw_final

,date,Keyword_1,Keyword_2,Keyword_3
0,2016-12-14,"(fog, 35)","(hot, 33)","(cold, 33)"
1,2016-12-15,"(fog, 31)","(hot, 29)","(sun, 28)"
2,2016-12-16,"(hot, 74)","(snow, 66)","(fog, 61)"
3,2016-12-19,"(hot, 74)","(snow, 66)","(fog, 61)"
4,2016-12-20,"(hot, 76)","(snow, 66)","(cold, 63)"
5,2016-12-21,"(hot, 47)","(sun, 42)","(fog, 42)"
6,2016-12-22,"(hot, 55)","(fog, 49)","(sun, 45)"
7,2016-12-23,"(hot, 50)","(fog, 45)","(sun, 44)"
8,2016-12-24,"(hot, 53)","(fog, 46)","(sun, 44)"
9,2016-12-25,"(hot, 55)","(fog, 48)","(sun, 45)"


# Rahul's Join - Build table with percentage for different keywords

In [72]:
# Unpivots tw_final
tw_final2 = pd.melt(tw_final, id_vars = ['date'])

# Rename variable for rank (1 being most popular)
tw_final2.rename(columns = {'variable':'rank'}, inplace = True)
tw_final2.head()

,date,rank,value
0,2016-12-14,Keyword_1,"(fog, 35)"
1,2016-12-15,Keyword_1,"(fog, 31)"
2,2016-12-16,Keyword_1,"(hot, 74)"
3,2016-12-19,Keyword_1,"(hot, 74)"
4,2016-12-20,Keyword_1,"(hot, 76)"


In [73]:
# Split value in tuple then build new columns
b = []
c = []

b.extend(a[1] for a in tw_final2['value'])
tw_final2['count'] = b

c.extend(a[0] for a in tw_final2['value'])
tw_final2['value'] = c

#tw_final2.head()

In [74]:
# Creates dataframe of total tweets for each day
df_count = dffinaltrained.groupby(['date']).count()

# Resets index
df_count.reset_index(level = 0, inplace = True)

df_count.rename(columns = {'text':'total_count'}, inplace = True)

#df_count.head()

In [75]:
# Build final table merged by Joining dataframes tw_final2 and df_count on date
merged = pd.merge(tw_final2, df_count, how='inner', on='date')

# Creates column for %daily word occurrence
merged['%daily word occurrence'] = (merged['count'] / merged['total_count']) * 100

# Rename column name Value for Key_word
merged.rename(columns = {'value':'key_word'}, inplace = True)

In [76]:
merged.head()

,date,rank,key_word,count,total_count,%daily word occurrence
0,2016-12-14,Keyword_1,fog,35,300,11.666667
1,2016-12-14,Keyword_2,hot,33,300,11.000000
2,2016-12-14,Keyword_3,cold,33,300,11.000000
3,2016-12-15,Keyword_1,fog,31,255,12.156863
4,2016-12-15,Keyword_2,hot,29,255,11.372549


In [78]:
merged.to_csv(path_or_buf="C:\\Data\\Twitter\\Tweets\\Daily_Word_Occurrence.csv", sep=',', na_rep='', \
               float_format=None, columns=['date', 'rank', 'key_word', 'count', 'total_count', '%daily word occurrence'], header=True, index=False, index_label=None, mode='w', encoding=None, \
               compression=None, quoting=None, quotechar='"', line_terminator='\n', chunksize=None, tupleize_cols=False, \
               date_format=None, doublequote=True, escapechar=None, decimal='.')
# Change header = false and mode = a from header = true and mode = w

# Plot final results with MetOffice data

# 1. Import MetOffice data and build join tables

In [238]:
# Plot options
# init_notebook: On jupyter
# plotly : on plotly

#init_notebook_mode()
import plotly
plotly.tools.set_credentials_file(username='Laura_Foulquier', api_key='zehu3fat3Mfs6v3pTNMY')

In [224]:
# Import the MetOffice Data
final_weather = pd.read_csv("C:\\Data\\Twitter\\Final_weather_for_plotting.csv", sep=',', delimiter=None)

#Rename Date column for futur join
final_weather.rename(columns = {'Date':'date'}, inplace = True)
final_weather.head()

,date,Temperature,Bucket_Weather
0,2016-12-07,11,2
1,2016-12-08,11,2
2,2016-12-09,12,2
3,2016-12-10,12,3
4,2016-12-11,7,1


In [225]:
# Join on dates with tweeter results
final_results = pd.merge(merged, final_weather, how='inner', on='date')

# Reduce the number of decimals to two for plotting 
for i in range(len(final_results.index)):
    final_results['%daily word occurrence'][i] = round(final_results['%daily word occurrence'][i], 2)

final_results.head()

C:\Users\Student29\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,date,rank,key_word,count,total_count,%daily word occurrence,Temperature,Bucket_Weather
0,2016-12-14,Keyword_1,fog,35,300,11.67,11,1
1,2016-12-14,Keyword_2,hot,33,300,11.00,11,1
2,2016-12-14,Keyword_3,cold,33,300,11.00,11,1
3,2016-12-15,Keyword_1,fog,31,255,12.16,9,2
4,2016-12-15,Keyword_2,hot,29,255,11.37,9,2


In [226]:
# Insert column for Weather depending on Bucket_Weather  number
final_results['Weather'] = None

for i in range(len(final_results.index)):
    if (final_results['Bucket_Weather'][i] == 1) == True:
        final_results['Weather'][i] = 'Sunny'

    elif (final_results['Bucket_Weather'][i] == 2) == True:
        final_results['Weather'][i] = 'Cloudy'
    
    elif (final_results['Bucket_Weather'][i] == 3) == True:
        final_results['Weather'][i] = 'Light Rain'
        
    elif (final_results['Bucket_Weather'][i] == 4) == True:
        final_results['Weather'][i] = 'Heavy Rain'

    elif (final_results['Bucket_Weather'][i] == 5) == True:
        final_results['Weather'][i] = 'Sleet / Hail'
        
    elif (final_results['Bucket_Weather'][i] == 6) == True:
        final_results['Weather'][i] = 'Snow'

    elif (final_results['Bucket_Weather'][i] == 7) == True:
        final_results['Weather'][i] = 'Thunder'

    elif (final_results['Bucket_Weather'][i] == 8) == True:
        final_results['Weather'][i] = 'Fog / Mist'

# Column with constant for plotting
final_results['plot'] = 16
final_results.head()

C:\Users\Student29\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Student29\Anaconda2\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Student29\Anaconda2\lib\site-packages\ipykernel\__main__.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Student29\Anaconda2\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning:


A value is trying to be set on a cop

,date,rank,key_word,count,total_count,%daily word occurrence,Temperature,Bucket_Weather,Weather,plot
0,2016-12-14,Keyword_1,fog,35,300,11.67,11,1,Sunny,16
1,2016-12-14,Keyword_2,hot,33,300,11.00,11,1,Sunny,16
2,2016-12-14,Keyword_3,cold,33,300,11.00,11,1,Sunny,16
3,2016-12-15,Keyword_1,fog,31,255,12.16,9,2,Cloudy,16
4,2016-12-15,Keyword_2,hot,29,255,11.37,9,2,Cloudy,16


# 2. Create different tables and arrays for plotting

In [227]:
# Create arrays and headers for plotting
# Header for plots
header = final_results.columns.values

npArray = np.array(final_results)
npArray.shape

(54L, 10L)

In [228]:
# Put the date in X, the rest of the parameter in Y
X = npArray[:,0]
Y = npArray[:,1:]

In [229]:
# DataFrame for rank = Keyword_1
rank1 = (final_results[(final_results['rank'] == 'Keyword_1')])
rank1 = rank1.reset_index(drop=True)
rank1 = rank1.drop(['count','total_count', 'Temperature', 'Bucket_Weather', 'Weather', ], axis=1)

npArray = np.array(rank1)
X1 = npArray[:,0]
Y1 = npArray[:,1:]

In [230]:
# DataFrame for rank = Keyword_2
rank2 = (final_results[(final_results['rank'] == 'Keyword_2')])
rank2 = rank2.reset_index(drop=True)
rank2 = rank2.drop(['count','total_count', 'Temperature', 'Bucket_Weather'], axis=1)

npArray = np.array(rank2)
X2 = npArray[:,0]
Y2 = npArray[:,1:]

In [231]:
# DataFrame for rank = Keyword_3
rank3 = (final_results[(final_results['rank'] == 'Keyword_3')])
rank3 = rank3.reset_index(drop=True)
rank3 = rank3.drop(['count','total_count', 'Temperature', 'Bucket_Weather'], axis=1)

npArray = np.array(rank3)
X3 = npArray[:,0]
Y3 = npArray[:,1:]

# 3. Plot temperature vs Keywords

In [239]:
# Plotting 

# Plotting the temperature
trace0 = go.Scatter(
    x = X,
    y = Y[:,5],
    name = 'Temperature at noon',
    yaxis='y2',
    line = dict(
        color = 'orange',
        width = 3
    )
)

# Plotting KeyWord1 occurence and name
trace1 = go.Bar(
    x = X1,
    y = Y1[:,2],
    text = Y1[:, 1],
    name = 'Occurence Keyword_1',
    marker = dict(
        color = 'rgb(255,100,100)',
        line = dict(
            color = 'rgb(255,0,0)',
            width = 1.5)
    )
)

# Plotting KeyWord2 occurence and name
trace2 = go.Bar(
    x = X2,
    y = Y2[:,2],
    text = Y2[:, 1],
    name = 'Occurence Keyword_2',
    marker = dict(
        color = 'rgb(158,202,225)',
        line = dict(
            color = 'rgb(8,48,107)',
            width = 1.5)
    )
)

# Plotting KeyWord3 occurence and name
trace3 = go.Bar(
    x = X3,
    y = Y3[:,2],
    text = Y3[:, 1],
    name = 'Occurence Keyword_3',
    marker = dict(
        color = 'rgb(100,250,250)',
        line = dict(
            color = 'rgb(100,200,250)',
            width = 1.5)
    )
)

layout = Layout(
    title = 'Keywords Occurence from tweets and temperature from MetOffice',
    xaxis = dict(
        title = 'Date',
        tickangle=320,
        autotick = 'False',
        ticks = 'outside'
    ),
    yaxis =dict(
        title='% of word occurence',
        range = [4,16],
        autotick = 'False',
        ticks = 'outside',
        tickfont=dict(
            color='rgb(148, 103, 189)',
        ) 
    ),
    yaxis2 = dict(
        title = 'Temperature',
        range = [0,20],
        autotick = 'False',
        ticks = 'outside',
        overlaying='y',
        side='right'
    ),
    showlegend = True,
    legend = dict(
         x = 0,
         y = -0.75
    ),
    font=dict(family='Old Standard TT, serif', size=14, color='purple')
)

data = [trace0, trace1, trace2, trace3]

fig = dict(data=data, layout = layout)
#iplot(fig)

# Put the plot in the plotly account
py.plot(fig, filename = 'Keywords Occurence from tweets and temperature from MetOffice')

u'https://plot.ly/~Laura_Foulquier/6'

# 4. Plot Actual Weather vs KeyWords

In [240]:
# Display the actual weather at noon from MetOffice
trace0 = go.Scatter(
    x = X,
    y = Y[:, -1],
    text = Y[:, -2],
    name = 'MetOffice Weather',
    mode = 'markers',
    marker = dict(
        size = 1,
        color = 'blue',
        opacity = 0.8
    )
)

# Plotting KeyWord1 occurence and name
trace1 = go.Bar(
    x = X1,
    y = Y1[:,2],
    text = Y1[:, 1],
    name = 'Occurence Keyword_1',
    marker = dict(
        color = 'rgb(255,100,100)',
        line = dict(
            color = 'rgb(255,0,0)',
            width = 1.5)
    )
)

# Plotting KeyWord2 occurence and name
trace2 = go.Bar(
    x = X2,
    y = Y2[:,2],
    text = Y2[:, 1],
    name = 'Occurence Keyword_2',
    marker = dict(
        color = 'rgb(158,202,225)',
        line = dict(
            color = 'rgb(8,48,107)',
            width = 1.5)
    )
)

# Plotting KeyWord3 occurence and name
trace3 = go.Bar(
    x = X3,
    y = Y3[:,2],
    text = Y3[:, 1],
    name = 'Occurence Keyword_3',
    marker = dict(
        color = 'rgb(100,250,250)',
        line = dict(
            color = 'rgb(100,200,250)',
            width = 0.5)
    )
)

layout = Layout(
    title = 'Keywords Occurence from tweets and actual weather from MetOffice',
    xaxis = dict(
        title = 'Date',
        tickangle=320,
        autotick = 'False',
        ticks = 'outside'
    ),
    yaxis =dict(
        title='% of word occurence',
        range = [0, 18],
        autotick = 'False',
        ticks = 'outside',
        tickfont=dict(
            color='rgb(148, 103, 189)',
        ) 
    ),
    showlegend = True,
    legend = dict(
         x = 0,
         y = -0.75
    ),
    font=dict(family='Old Standard TT, serif', size=14, color='purple')
)

data = [trace0, trace1, trace2, trace3]

fig = dict(data=data, layout = layout)
#iplot(fig)

# Put the plot in the plotly account
py.plot(fig, filename = 'Keywords Occurence from tweets and actual weather from MetOffice')

u'https://plot.ly/~Laura_Foulquier/8'